# 列舉所有資產的riskiness r

In [1]:
import sympy
from pandas import Series,DataFrame
from scipy.optimize import fsolve
import math
import time
import pandas as pd
import numpy  as  np
import os

# 無風險利率
* 我們須扣除無風險利率，得到超額報酬率以計算Riskiness r
* 目前使用LIBOR [1個月利率](https://www.global-rates.com/interest-rates/libor/american-dollar/usd-libor-interest-rate-overnight.aspx)

In [2]:
rfrate = 2.45785 / 12 /100
rfrate

0.0020482083333333333

# 關於超額報酬的數量級:
* 將要帶入公式的數值調整到介在0-100間，目的是為了讓大部分的Riskiness r介在0-100間。

In [3]:
def f1(x,arr_returns):
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)

In [4]:
def get_riskiness_r(guess , arr_returns):
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

# 計算Riskiness R，並依照R值排序，傳回前十名的ETF

In [5]:
def get_top10_etf(data_file):
    # 讀檔
    #data_file = '../../HW1/ETF_data_set/return_data/Alternatives ETF List (35)/Alternatives ETF List (35)_return.csv';
    df = pd.read_csv(data_file)
    etf_list = list(df.columns)[1:]
    df = df.iloc[:,1:]
    
    # 計算每支ETF的riskiness r
    etf_dic = {}
    for etf_name in etf_list:
        # 過濾平均值正負
        #if df[etf_name].mean(axis=0) < 0:
        #    continue
        arr_returns = df[etf_name] - rfrate
        #arr_returns = arr_returns*1000      #調整數量級
        guess = 10**(-5)
        #### 列出細項
        #print('%s指標值:  %f' %(etf_name,risk2[0]) )
        etf_dic[etf_name] = get_riskiness_r(guess,arr_returns)
    
    # ETF依照riskiness r排序
    sorted_x = sorted(etf_dic.items(), key=lambda kv: kv[1])
        #sorted_reversed_x = list(reversed(sorted_x))
    
    # 取前10名
    top10 = sorted_x[:10]
        # top10 = sorted_reversed_x[:10]
    #print(top10)
    
    # 取前10的ETF名稱
    etf_top10 = [etf[0] for etf in top10]
    
    #前10的平均
    top10_average = df[etf_top10].mean(axis=1)
    
    #print(etf_top10)
    return etf_top10 , top10_average

# 讀取全部ETF資料夾

In [6]:
data_path = '../../HW1/ETF_data_set/return_data/'
allgroup_top10_return = DataFrame()
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    if os.path.exists(subdir_path) and os.path.isdir(subdir_path):
        file = subdir + '_return.csv'
        file_path = os.path.join(subdir_path, file)
        #print(file_path)
        if os.path.isfile(file_path):
            # 提出[前10名的名稱,前10名的平均各期報酬]
            [ top10_etf,allgroup_top10_return[subdir] ] = get_top10_etf(file_path)
            print('Group:',subdir)
            print('Top10:',top10_etf)
            print('================')

D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Group: Alternatives ETF List (35)
Top10: ['MOM', 'RLY', 'CHEP', 'VAMO', 'FTLS', 'SIZ', 'QAI', 'WTMF', 'MCRO', 'HDG']
Group: Broad Asia ETF List (44)
Top10: ['AXJL', 'IQDE', 'HEEM', 'DBAP', 'AIA', 'GEM', 'IEMG', 'EEMA', 'XSOE', 'AAXJ']
Group: Commodity ETF List (125)
Top10: ['DBA', 'BCM', 'RJZ', 'DBB', 'UCI', 'RJI', 'FTGC', 'GCC', 'KOLD', 'BOIL']
Group: Consumer Discretionary Equity ETF List (31)
Top10: ['RETL', 'FDIS', 'IYC', 'VCR', 'XLY', 'RTH', 'INCO', 'RXI', 'PSCD', 'UCC']
Group: Consumer Staples Equity ETF List (26)
Top10: ['WBIE', 'PSL', 'PSCC', 'VDC', 'EMCG', 'FSTA', 'RHS', 'KXI', 'IYK', 'UGE']
Group: Crude Oil ETF List (22)
Top10: ['UCO', 'DTO', 'SCO', 'USL', 'SZO', 'OLEM', 'OLO', 'DBO', 'USO']
Group: Currency ETF List (36)
Top10: ['FXCH', 'FXS', 'FXA', 'CEW', 'DBV', 'FXC', 'EUFX', 'USDU', 'UUP', 'FXF']
Group: Developed Asia Pacific ETF List (108)
Top10: ['DOL', 'JPMV', 'EWV', 'DVYA', 'DTH', 'IDOG', 'DOO', 'JPNL', 'ENZL', 'HEWJ']
Group: Developed Markets ETF List 1 (100)
Top10: 

In [7]:
allgroup_top10_return.iloc[:,0]

0    -0.014848
1     0.004294
2     0.017014
3     0.001131
4    -0.004865
5     0.000849
6     0.010138
7    -0.004461
8     0.006556
9    -0.009867
10    0.015666
11    0.005291
12   -0.003252
13    0.004224
14   -0.006747
15    0.000585
16   -0.005576
17    0.002532
18    0.007081
19    0.000081
20    0.013821
21    0.007662
22    0.004123
23    0.010876
24    0.017250
25   -0.020208
26   -0.006661
27    0.010356
28    0.009172
29   -0.013563
30    0.007572
31    0.005089
32   -0.006795
33   -0.035729
34   -0.001529
35   -0.034942
Name: Alternatives ETF List (35), dtype: float64

In [33]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    v = 0
    for i in range(len(w)):
        v = v + w[i]*risk(w[i]*(all_return.iloc[:,i]))
    return v

def penalty(w):
    if min(w)<0 or max(w)>1:
        return 100000
    else:
        return 0
def portretmean(weight,all_return):
    w = weight
    a = all_return
    return np.mean(np.dot(np.array(all_return),np.array(weight)))    

def bestweight(w):
    r = []
    r.append(penalty(w)+np.abs(sum(w)-1))
    for i in range(len(w)-1):
        v =  np.abs( risk(w[i]*(all_return.iloc[:,i]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i])-rfrate)/(portretmean(w,all_return)-rfrate) ) \
            + np.abs( risk(w[i+1]*(all_return.iloc[:,i+1]))/ sumrisk(w,all_return)  \
            - (np.mean(all_return.iloc[:,i+1])-rfrate)/(portretmean(w,all_return)-rfrate))
        r.append(v) 
    print('權重:',w,'誤差值:',v+penalty(w)+np.abs(sum(w)-1))
    return r

In [9]:
def fourdrop(x):
    a = []
    for i in range(len(x)):
        a.append(round(sum(x[i]),3))
    return a
x = np.array(allgroup_top10_return.iloc[:,[0]])
x = fourdrop(x)
y = np.array(allgroup_top10_return.iloc[:,[1]])
y = fourdrop(y)
all_return = pd.DataFrame([x,y]).T
all_return = all_return

In [10]:
x = np.array([-0.01,0,0.0258])
y = np.array([-0.05,0,0.134])
all_return = pd.DataFrame([x,y]).T

In [39]:
a = list(range(0, 7))
all_return = allgroup_top10_return.iloc[:,a]
n = all_return.shape[1]
#weight = [0.4,0.6]
weight = [1/n]*n
r2 = fsolve(bestweight, weight)
print(r2)

D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)
D:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959401830814
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959401830814
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959401830814
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959410413197
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959184662707
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959357155638
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959188258305
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959300601887
權重: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714] 誤差值: 15.269959421311881
權重: [0.14285714 0.14285714 0